In [2]:
import pandas as pd, geopandas as gpd
import calendar, time
import re
import json

In the first installment of this scripts, the system failed to geolocate some of messages due to incorrect parsing

In some instances, double posting or posting with the suffixes "g.r" or "g" was interpreted as a city, whereas these are abbreviations to "birthdate"

These ended up being erroneously geocoded by Nominatim

We'll first remove these results from the output dataset

Add a fail-safe to prevent geocoding birthdates instead of places

Re-geocode the failed geocoded messages by keeping their ids and re-parsing them


In [3]:
with open("../geolocated_datasets/1686571967-geolocated-@poisk_in_ua.json", encoding="utf-8-sig") as filec:
    original_gdf = gpd.read_file(filec, driver="GeoJSON")
timestamp = calendar.timegm(time.gmtime())

In [4]:
located_cities = original_gdf.loc[original_gdf["tag"]=="city"]

In [5]:
def find_dates(source):
    return re.search("\d{2}\.\d{2}\.\d{4}", source)


In [6]:
_test = located_cities
_test["test"] = located_cities["location"].apply(lambda a: find_dates(a))

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [7]:
identified_failures = _test.loc[_test["test"].notnull()]
identified_failures = identified_failures.drop("test", axis="columns")
identified_failures_1 = _test.loc[_test["message"].str.contains("Не идентифицирован", case = False, na = False)]

In [8]:
original_gdf["id"] = original_gdf["id"].astype(int)
identified_failures["id"] = identified_failures["id"].astype(int)
original_gdf["date"] = original_gdf["date"].astype(str)
identified_failures["date"] = identified_failures["date"].astype(str)
identified_failures_1["id"] = original_gdf["id"].astype(int)
identified_failures_1["date"] = identified_failures["date"].astype(str)

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [9]:
cleaned = original_gdf.drop(identified_failures.index, axis=0)
cleaned = test.drop(identified_failures_1.index, axis=0)

In [10]:
output_gdf_filename = f"../geolocated_datasets/{str(timestamp)}-cleaned-of-birthdates-@poisk_in_ua.json"
output_file = open(output_gdf_filename, "w", encoding="utf-8-sig")
contents = cleaned.to_json()
output_file.write(contents)

11109823

In [12]:
identified_failures

,id,location,tag,date,_latitude,_longitude,message,geometry
12910,20383,28.10.1978 р,city,2023-04-20 16:20:58+00:00,-21.6423423,-47.4661723,Факаев Айдар Расимович\n28.10.1978 г.р\nг.Нефт...,POINT (-47.46617 -21.64234)
12934,16881,08.08.1981 .,city,2023-03-16 10:05:56+00:00,-45.7923048,-67.5051351,Шарапов Руслан Варисович\n08.08.1981 г..р.\nг....,POINT (-67.50514 -45.79230)
12948,15115,19.11.1986 р,city,2023-03-01 07:25:32+00:00,-40.8303727,-63.02291346896551,Таурит Ян Вячеславович\n19.11.1986 г.р\nг.Саян...,POINT (-63.02291 -40.83037)
12957,14425,30.12.1990,city,2023-02-23 13:40:00+00:00,29.6192729,52.4677027,Гальцов Ярослав Игоревич\n30.12.1990 г. р.\nг....,POINT (52.46770 29.61927)
12958,14160,03.08.1988,city,2023-02-21 11:20:00+00:00,-6.2195943,106.8454693,Шакиров Евгений Юрьевич\n03.08.1988 г. р.\nг. ...,POINT (106.84547 -6.21959)
12975,12131,28.09.1973 р,city,2023-02-01 12:05:37+00:00,40.00976655555556,-74.14628427777778,Алексиевич Евгений Викторович\n28.09.1973 г.р\...,POINT (-74.14628 40.00977)
12977,12053,29.10.1997. р,city,2023-01-31 15:55:17+00:00,-40.83287101632653,-63.0261349510204,Мазуров Богдан Федорович \n29.10.1997. г. р\nг...,POINT (-63.02613 -40.83287)
12978,11660,28.03.1985 р,city,2023-01-25 07:20:23+00:00,40.00988522222222,-74.1465926111111,Старший лейтенант Шевелев Юрий Владимирович \n...,POINT (-74.14659 40.00989)
12997,10061,09.09.1983 р,city,2022-12-27 09:59:59+00:00,-4.3762488,15.2579302,Носов Евгений Евгеньевич \n09.09.1983 г.р\nЧВК...,POINT (15.25793 -4.37625)
13016,7883,13.12.1993,city,2022-11-19 08:35:30+00:00,29.5689945,52.5557341,#поиск\n\nДобрый вечер. Я каждый день просматр...,POINT (52.55573 29.56899)


In [11]:
identified_failures_name = f"../to_geocode_datasets/{timestamp}-to_reparse.json"
output_file_1 = open(identified_failures_name, "w", encoding="utf-8-sig")
contents = identified_failures.to_json()
output_file_1.write(contents)

20331